In [14]:
from parsi_io.modules.number_extractor import NumberExtractor
extractor = NumberExtractor()
res = extractor.run('پانزده بیست و پنجم ')
res[0]['value'] / res[1]['value']

0.6

In [3]:
extractor.run('دو هزار و ششصد')

[{'span': [0, 14], 'phrase': 'دو هزار و ششصد', 'value': 2600}]

In [4]:
extractor.run('سی و چهار صدم')

[{'span': [0, 13], 'phrase': 'سی و چهار صدم', 'value': 30.04}]

In [12]:
extractor.run('دوم ')

[{'span': [0, 2], 'phrase': 'دو', 'value': 2}]